In [ ]:
import pandas
import datetime
import numpy
import urllib
import os
import calendar
import requests
import io

In [ ]:
# Setup a start date and end date 
# between which to download data
startDateDwnld = datetime.datetime( 2012, 9, 1 )
endDateDwnld = datetime.datetime( 2014, 12, 31 )
# Setup output directory and file
outBaseDir = "/home/bharat/Documents/code/rbsp-geo-pos/"
opFileSatA = outBaseDir + "/rbspPosSatA.txt"
opFileSatB = outBaseDir + "/rbspPosSatB.txt"
# Before appending to the file! delete it if it 
# exists already. We dont want to append to old data!
if os.path.isfile( opFileSatA ):
    os.remove( opFileSatA )
if os.path.isfile( opFileSatB ):
    os.remove( opFileSatB )
# Setup necessary urls
# SampleUrl = "http://rbspgway.jhuapl.edu/rTools/orbitlist/lib/php/orbitlist.php?cli=GEO%20RBSP_A%201346457600%2086400%20-hdr%20-r"
baseUrl = "http://rbspgway.jhuapl.edu/rTools/" +\
        "orbitlist/lib/php/orbitlist.php?"
# get the difference in days and loop through 
# each day to download the ascii file!
# We'll download the full day
delDates = endDateDwnld - startDateDwnld
if delDates.days < 1:
    loopDays = 1
else:
    loopDays = delDates.days
for ld in range(loopDays + 1):
    # get the timestamp of each date
    currDate = startDateDwnld + datetime.timedelta(days=ld)
    currTimestamp = calendar.timegm(currDate.timetuple())
    # We'll do a urlencode
    urlDictSatA = { 'cli' : 'GEO RBSP_A ' + \
                   str(currTimestamp) + ' 86400 -hdr -r' }
    urlDictSatB = { 'cli' : 'GEO RBSP_B ' + \
                   str(currTimestamp) + ' 86400 -hdr -r' }
    currUrlSatA = baseUrl + urllib.urlencode(urlDictSatA)
    currUrlSatB = baseUrl + urllib.urlencode(urlDictSatB)
    # Get data in a DF for both SAT-A and SAT-B
    # We'll get datetime in python datetime format
    # Its easy to manage things that way!
    reqObj = requests.get(currUrlSatA).content
    currDFSatA = pandas.read_csv( \
                        io.StringIO(reqObj.decode('utf-8')),\
                        delim_whitespace=True,\
                        parse_dates=[[0, 1, 2, 3, 4, 5]], \
                        date_parser=lambda *columns: datetime.datetime(*map(int, columns) ) )
    currDFSatA.columns = [ "datetime", "alt", "lat", "lon" ]
    currDFSatA["dateStr"] = [ x.strftime("%Y%m%d") for x in currDFSatA["datetime"] ]
    currDFSatA["timeStr"] = [ x.strftime("%H%M") for x in currDFSatA["datetime"] ]
    reqObj = requests.get(currUrlSatB).content
    currDFSatB = pandas.read_csv( \
                        io.StringIO(reqObj.decode('utf-8')),\
                        delim_whitespace=True, \
                        parse_dates=[[0, 1, 2, 3, 4, 5]], \
                        date_parser=lambda *columns: datetime.datetime(*map(int, columns) ) )
    currDFSatB.columns = [ "datetime", "alt", "lat", "lon" ]
    currDFSatB["dateStr"] = [ x.strftime("%Y%m%d") for x in currDFSatB["datetime"] ]
    currDFSatB["timeStr"] = [ x.strftime("%H%M") for x in currDFSatB["datetime"] ]
    
    # Append the data in a text file
    with open(opFileSatA, 'a') as opfA:
        currDFSatA.to_csv(opfA, header=False,\
                          index=False, sep=' ',\
                          columns=[ "dateStr", "timeStr", "alt", "lat", "lon" ] )
    with open(opFileSatB, 'a') as opfB:
        currDFSatB.to_csv(opfB, header=False,\
                          index=False, sep=' ',\
                          columns=[ "dateStr", "timeStr", "alt", "lat", "lon" ] )